# In this code we will perform the data ingestion and manipulation that will be later used in the modular project

In [1]:
# Importing core libraries

import os 
import sys

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\agost\\Data_Career\\End_to_end_projects\\Abalone_Age_Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionManipulationConfig:
    root_dir: Path
    local_data_file: Path
    source_URL: str
    save_training_file: Path
    target_col: str

In [5]:
from Abalone_Age_Prediction.utils.common import create_directories, read_yaml, save_data
from Abalone_Age_Prediction.constants import *
from Abalone_Age_Prediction import logger

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) #the artifacts_root is the key of the dictionary created
                                                # in the yaml file and we can read this key like that instead of
                                                # ["artifacts_root"] because we used the ConfigBox in the common.py file


    def get_data_ingestion_config(self) -> DataIngestionManipulationConfig:
        config = self.config.data_ingestion_manipulation #data ingestion is the other key value of the dictionary in the config.yaml file

        create_directories([config.root_dir,config.save_training_file])

        data_ingestion_manipulation_config = DataIngestionManipulationConfig(
            root_dir=config.root_dir,
            local_data_file = config.local_data_file,
            source_URL = config.source_URL,
            save_training_file= config.save_training_file,
            target_col=self.params.TARGET
        )                                     

        return data_ingestion_manipulation_config

In [7]:
import urllib.request as request

In [8]:
class DataIngestionManipulation:
    def __init__(self, config: DataIngestionManipulationConfig):
        self.config = config

    def download_file(self):
        """
        file_path: str
        Download, if it doesn't already exists, the csv file with data, don't need a return, just to save the Data
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )

            #logger.info(f"{filename} is downloading with the following info: \n{headers}")
        else:
            logger.info(f"File already downloaded")
        
        return None
    
    def read_file(self):
        """
        Read the csv file
        """
        try:
            
            df = pd.read_csv(self.config.local_data_file)

            return df

        except Exception as e:
            e

    def preprocess_file(self,df):
        """
        Preprocess the data and save it into a training and testing files
        """

        numerical_cols = df.select_dtypes(include='number').columns
        categorical_cols = df.select_dtypes(include='object').columns

        

        for col in categorical_cols:
            encoder = LabelEncoder()
            df[col] = encoder.fit_transform(df[col])

        X = df.drop(labels = self.config.target_col, axis = 1)
        y = df[self.config.target_col]
        
        scaler = MinMaxScaler()
        numerical_cols = numerical_cols.drop(self.config.target_col)
        X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=10)

        train_data = pd.concat([X_train,y_train])
        test_data = pd.concat([X_test,y_test])

        training_path = self.config.save_training_file
        #print("Before")
        save_data(Path(training_path),train_data,"train.csv")
        #print("After")
        save_data(Path(training_path),test_data,"test.csv")
        
        return None
    

    


In [9]:
#os.chdir("../../")

In [12]:

%pwd

'c:\\Users\\agost\\Data_Career\\End_to_end_projects\\Abalone_Age_Prediction'

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_manipulation_config = config.get_data_ingestion_config()
    data_ingestion_manipulation = DataIngestionManipulation(config=data_ingestion_manipulation_config)
    data_ingestion_manipulation.download_file()
    data = data_ingestion_manipulation.read_file()
    data_ingestion_manipulation.preprocess_file(data)
    
except Exception as e:
    raise e

[2024-06-13 19:56:13,321: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-13 19:56:13,322: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-13 19:56:13,323: INFO: common: created directory at: artifacts]
[2024-06-13 19:56:13,323: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-13 19:56:13,324: INFO: common: created directory at: artifacts/training]
[2024-06-13 19:56:13,581: INFO: 983651373: artifacts/data_ingestion/abalone.csv is downloading with the following info: 
Connection: close
Content-Length: 191962
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: text/plain; charset=utf-8
ETag: "7297242ad999990e87c9e91ce42e50c9920611902ef8e3289192993e1092cb91"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4580:2A87CD:15CAA8:16B1FD:666B227F
Accept-Ranges: